In [9]:
import os
import numpy as np
from pathlib import Path
import torch
from torch import nn


In [10]:
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

bindata = {}
for fp in sorted(Path("data/ManipNetBIN").iterdir()):
    # print(fp)
    bindata[fp.stem] = np.fromfile(fp, dtype=np.float32)
for k, v in bindata.items():
    print(k, v.shape)

os.chdir("data/ManipNetBIN")
print(os.getcwd())
# num = np.fromfile('Decoder_b0.bin')
# print(num)

/workspace/rendering/data
/workspace/rendering
Decoder_b0 (214,)
Decoder_w0 (328704,)
DenseRes0_b0 (1536,)
DenseRes0_w0 (2359296,)
DenseRes1_b0 (1536,)
DenseRes1_w0 (2359296,)
DenseRes2_b0 (1536,)
DenseRes2_w0 (2359296,)
DenseRes3_b0 (1536,)
DenseRes3_w0 (2359296,)
Encoder0_b0 (512,)
Encoder0_w0 (98304,)
Encoder1_b0 (512,)
Encoder1_w0 (172032,)
Encoder2_b0 (512,)
Encoder2_w0 (932864,)
Xmean (2350,)
Xstd (2350,)
Ymean (214,)
Ystd (214,)
/workspace/rendering/data/ManipNetBIN


In [79]:
class ResBlock():
    def __init__(self, in_vec, out_vec):
        super(ResBlock, self).__init__()
        self.FC0= nn.Linear(in_vec, out_vec)
        self.FC1 = nn.Linear(in_vec, out_vec)
    def __call__(self, H_zero):
        H_zero = nn.ReLU(H_zero)
        H_one = self.FC0(H_zero) + H_zero
        H_one  = nn.ReLU(H_one)
        H_two = self.FC1(H_one) + H_one + H_zero
        return H_two


In [87]:
# 訓練に際して、可能であればGPU（cuda）を設定します。GPUが搭載されていない場合はCPUを使用します
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# modelを定義します
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.Decoder = nn.Linear(1536, 214)
        self.flatten = nn.Flatten()
        self.Encoder0= nn.Linear(192, 512)
        self.Encoder1 = nn.Linear(336, 512)
        self.Encoder2 = nn.Linear(1822, 512)
        # self.res_fc = nn.Linear(1536, 1536)
        self.DenseRes0 = ResBlock(1536, 1536)
        self.DenseRes1 = ResBlock(1536, 1536)

    def forward(self, pose, traj, sensor):
        x = self.flatten(x)
        pose = self.Encoder0(pose)
        traj = self.Encoder1(traj)
        sensor = self.Encoder2(sensor)
        out = torch.cat(pose, traj, sensor, dim=1)
        out = self.DenseRes0(out)
        out = self.DenseRes1(out)
        out = self.Decoder(out)
        return out

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (Decoder): Linear(in_features=1536, out_features=214, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (Encoder0): Linear(in_features=192, out_features=512, bias=True)
  (Encoder1): Linear(in_features=336, out_features=512, bias=True)
  (Encoder2): Linear(in_features=1822, out_features=512, bias=True)
)


In [61]:
for name, param in model.named_parameters():
    print(name, param.shape)

Decoder.weight torch.Size([214, 1536])
Decoder.bias torch.Size([214])
Encoder0.weight torch.Size([512, 192])
Encoder0.bias torch.Size([512])
Encoder1.weight torch.Size([512, 336])
Encoder1.bias torch.Size([512])
Encoder2.weight torch.Size([512, 1822])
Encoder2.bias torch.Size([512])
res_fc.weight torch.Size([1536, 1536])
res_fc.bias torch.Size([1536])


In [14]:
for k, v in bindata.items():
    print(k, v.shape)

Decoder_b0 (214,)
Decoder_w0 (328704,)
DenseRes0_b0 (1536,)
DenseRes0_w0 (2359296,)
DenseRes1_b0 (1536,)
DenseRes1_w0 (2359296,)
DenseRes2_b0 (1536,)
DenseRes2_w0 (2359296,)
DenseRes3_b0 (1536,)
DenseRes3_w0 (2359296,)
Encoder0_b0 (512,)
Encoder0_w0 (98304,)
Encoder1_b0 (512,)
Encoder1_w0 (172032,)
Encoder2_b0 (512,)
Encoder2_w0 (932864,)
Xmean (2350,)
Xstd (2350,)
Ymean (214,)
Ystd (214,)


In [72]:
netlist = ["Decoder"]
net_name_list = []
# for i in range(4):
#     netlist.append("DenseRes" + str(i))
for i in range(3):
    netlist.append("Encoder" + str(i))
print(netlist)
for name in netlist:
    net_name_list.append([name + ".weight", name + "_w0"])
    net_name_list.append([name + ".bias", name + "_b0"])

shape_list = [[214, 1536], [214], [512, 192], [512], [512, 336], [512], [512, 1822], [512]]
for i in range(len(net_name_list)):
    net_name_list[i].append(shape_list[i])
print(net_name_list)



['Decoder', 'Encoder0', 'Encoder1', 'Encoder2']
[['Decoder.weight', 'Decoder_w0', [214, 1536]], ['Decoder.bias', 'Decoder_b0', [214]], ['Encoder0.weight', 'Encoder0_w0', [512, 192]], ['Encoder0.bias', 'Encoder0_b0', [512]], ['Encoder1.weight', 'Encoder1_w0', [512, 336]], ['Encoder1.bias', 'Encoder1_b0', [512]], ['Encoder2.weight', 'Encoder2_w0', [512, 1822]], ['Encoder2.bias', 'Encoder2_b0', [512]]]


In [86]:
with torch.no_grad():
    for name, param in model.named_parameters():
        print(name)
        for k, v in bindata.items():
            for i in range(len(net_name_list)):
                # print(net_name_list[i][0], name)
                if name == net_name_list[i][0] and k == net_name_list[i][1]:
                    print(param.shape, name, k)
                    rephape_weight = torch.reshape(torch.from_numpy(bindata[net_name_list[i][1]]), net_name_list[i][2])
                    param.copy_(rephape_weight)
        
            # resblock 
            if name == "DenseRes0_b0":
                model.res1.FC.parameters.copy()
    
print(model.Encoder0)
print(model.DenceRes0)

Decoder.weight
torch.Size([214, 1536]) Decoder.weight Decoder_w0
Decoder.bias
torch.Size([214]) Decoder.bias Decoder_b0
Encoder0.weight
torch.Size([512, 192]) Encoder0.weight Encoder0_w0
Encoder0.bias
torch.Size([512]) Encoder0.bias Encoder0_b0
Encoder1.weight
torch.Size([512, 336]) Encoder1.weight Encoder1_w0
Encoder1.bias
torch.Size([512]) Encoder1.bias Encoder1_b0
Encoder2.weight
torch.Size([512, 1822]) Encoder2.weight Encoder2_w0
Encoder2.bias
torch.Size([512]) Encoder2.bias Encoder2_b0
Linear(in_features=192, out_features=512, bias=True)


AttributeError: 'NeuralNetwork' object has no attribute 'DenceRes0'

<bound method Module.parameters of Linear(in_features=1536, out_features=1536, bias=True)>
